# Bert4Rec


Постановка задачи: В последовательных рекомендательных системах мы прогнозируем следующий элемент (например, фильм) для пользователя на основе последовательности его прошлых взаимодействий. То есть у каждого пользователя имеется упорядоченный список просмотренных фильмов (история), и система должна рекомендовать, что он посмотрит дальше. Традиционные подходы к последовательным рекомендациям используют рекуррентные нейросети или однонаправленные трансформеры, которые просматривают историю слева направо, предсказывая следующий элемент по предыдущим. Однако такие однонаправленные модели имеют ограничения: (a) они учитывают только левый контекст (прошлое) и игнорируют правый контекст, что может приводить к неоптимальному представлению последовательности; (b) они жёстко предполагают строго упорядоченную последовательность, что не всегда соответствует реальности поведения пользователей


Выбор модели: Для устранения этих ограничений была предложена модель BERT4Rec – Bidirectional Encoder Representations from Transformers for Sequential Recommendation. По аналогии с BERT в NLP, BERT4Rec применяет глубокое двунаправленное самовнимание (Transformer) для моделирования последовательности действий пользователя. В отличие от однонаправленных моделей (например, SASRec на основе Transformer Decoder), BERT4Rec одновременно учитывает и левый, и правый контекст каждого элемента в истории. Это позволяет каждому элементу истории «впитать» информацию от соседей с обеих сторон, формируя более информативное представление предпочтений пользователя. Таким образом, модель не делает жёсткого предположения о порядке и может улавливать произвольные дальние зависимости в последовательности благодаря механизму самовнимания. BERT4Rec демонстрирует устойчиво высокие результаты качества рекомендаций по сравнению с предыдущими моделями последовательных рекомендаций


Ключевая идея – предсказание маскированных элементов: Прямое обучение двунаправленной модели сталкивается с проблемой тривиальности – если модель увидит всю последовательность, включая будущие элементы, то предсказывать следующий не имеет смысла. Поэтому в BERT4Rec применяется задача Cloze (маскированного моделирования), аналогичная masked language modeling в BERT. Во время тренировки некоторые элементы последовательности случайно маскируются специальным токеном [MASK], и модель учится восстанавливать маскированные позиции по их левому и правому контексту. Например, последовательность из пяти фильмов [v1, v2, v3, v4, v5] может быть преобразована в [v1, [mask], v3, [mask], v5], и модель должна предсказать, что скрывается за первым [mask] (это v2) и за вторым [mask] (v4). Поскольку маскирование может затрагивать несколько позиций, одна последовательность даёт сразу несколько обучающих примеров, что делает обучение эффективнее, чем предсказание только следующего элемента


Инференс (рекомендации): После обучения модель способна генерировать персональные рекомендации. Чтобы предсказать следующий фильм для пользователя, к концy известной истории пользователя добавляют токен [MASK] и пропускают последовательность через модель. Модель, опираясь на двунаправленный контекст, выдаёт распределение вероятностей на месте этого маскированного токена – какие фильмы наиболее вероятно окажутся следующими. Рекомендуется фильм с наивысшей вероятностью, а для метрик качества обычно рассматривают топ-K (например, топ-10) наиболее вероятных кандидатов. Такой подход позволяет непосредственно использовать обученную двунаправленную модель для предсказания будущих взаимодействий пользователя


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from rectools.metrics import Recall, Precision, NDCG, calc_metrics
from rectools.dataset import Dataset as RecDataset

# ╔═══════════════════════════════════════════════════════════════════════╗
# ║                         МОДЕЛЬ BERT4REC                               ║
# ╚═══════════════════════════════════════════════════════════════════════╝

class BERT4Rec(nn.Module):
    """
    BERT4Rec - адаптация архитектуры BERT для задачи последовательных рекомендаций.
    
    Основная идея: используем bidirectional self-attention для понимания контекста
    последовательности действий пользователя и предсказания замаскированных товаров.
    """
    
    def __init__(self, num_items, max_seq_len=50, embed_dim=64, num_heads=2,
                 hidden_dim=256, num_layers=2, dropout=0.1):
        """
        Инициализация модели BERT4Rec
        
        Параметры:
        ----------
        num_items : int
            Количество уникальных товаров в каталоге
        max_seq_len : int
            Максимальная длина последовательности действий пользователя
        embed_dim : int  
            Размерность эмбеддингов (векторных представлений)
        num_heads : int
            Количество голов внимания в multi-head attention
        hidden_dim : int
            Размерность скрытого слоя в feedforward network
        num_layers : int
            Количество слоев трансформера
        dropout : float
            Вероятность отключения нейронов для регуляризации
        """
        super(BERT4Rec, self).__init__()

        # ===== СЛОЙ ЭМБЕДДИНГОВ ТОВАРОВ =====
        self.item_embedding = nn.Embedding(num_items + 2, embed_dim, padding_idx=0)
        """
        Таблица эмбеддингов размера (num_items + 2) × embed_dim
        
        Индексация:
        - 0: [PAD] токен (padding) - всегда нулевой вектор
        - 1...num_items: реальные товары из каталога
        - num_items + 1: [MASK] токен для маскированного обучения
        
        padding_idx=0 гарантирует, что эмбеддинг PAD токена не обновляется при обучении
        """
        
        # ===== ПОЗИЦИОННЫЕ ЭМБЕДДИНГИ =====
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        """
        Таблица позиционных эмбеддингов max_seq_len × embed_dim
        
        Зачем нужны:
        - Трансформер не имеет встроенного понимания порядка элементов
        - Позиционные эмбеддинги добавляют информацию о позиции каждого товара
        - Кодирует позиции от 0 до max_seq_len-1
        - Позволяет модели различать "купил вначале" и "купил в конце"
        """

        # ===== ЭНКОДЕР ТРАНСФОРМЕРА =====
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True,
            activation='gelu'
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        """
        Стек из num_layers слоев трансформера
        
        Каждый слой содержит:
        1. Multi-head self-attention (с num_heads головами)
           - Позволяет каждому товару "смотреть" на все остальные
        2. Layer normalization
        3. Position-wise feedforward network:
           - Linear(embed_dim → hidden_dim)
           - GELU activation
           - Linear(hidden_dim → embed_dim)
        4. Еще один layer normalization
        5. Residual connections вокруг обоих подслоев
        
        GELU (Gaussian Error Linear Unit) - более гладкая версия ReLU
        """

        # ===== ВЫХОДНОЙ СЛОЙ =====
        self.output_layer = nn.Linear(embed_dim, num_items + 1)
        """
        Преобразует скрытые представления в логиты для каждого товара
        
        - Входная размерность: embed_dim
        - Выходная размерность: num_items + 1 (без PAD токена)
        - Для каждой позиции предсказывает scores всех возможных товаров
        """

    def forward(self, input_ids):
        """
        Прямой проход модели
        
        Параметры:
        ----------
        input_ids : torch.Tensor
            Последовательности товаров, shape: [batch_size, seq_len]
            
        Возвращает:
        -----------
        logits : torch.Tensor
            Логиты для каждой позиции, shape: [batch_size, seq_len, num_items+1]
        """
        
        # Получаем device и размеры
        device = input_ids.device
        batch_size, seq_len = input_ids.size()

        # ===== СОЗДАНИЕ ПОЗИЦИОННЫХ ИНДЕКСОВ =====
        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, seq_len)
        """
        Пошагово:
        1. torch.arange(seq_len) → [0, 1, 2, ..., seq_len-1]
        2. .unsqueeze(0) → [[0, 1, 2, ...]] (добавляем batch dimension)
        3. .expand(batch_size, seq_len) → копируем для каждого примера в батче
        
        Результат: матрица позиций для всего батча
        """

        # ===== КОМБИНИРОВАНИЕ ЭМБЕДДИНГОВ =====
        x = self.item_embedding(input_ids) + self.position_embedding(positions)
        """
        Складываем два типа эмбеддингов:
        - Эмбеддинги товаров: "что это за товар?"
        - Позиционные эмбеддинги: "где он находится в последовательности?"
        
        Результат: [batch_size, seq_len, embed_dim]
        """

        # ===== МАСКА ПАДДИНГА =====
        padding_mask = (input_ids == 0)
        """
        Булева маска: True = позиция с PAD токеном
        Трансформер будет игнорировать эти позиции при вычислении attention
        Это важно для корректной работы с последовательностями разной длины
        """

        # ===== ПРОХОЖДЕНИЕ ЧЕРЕЗ ЭНКОДЕР =====
        x = self.encoder(x, src_key_padding_mask=padding_mask)
        
        # ===== ГЕНЕРАЦИЯ ЛОГИТОВ =====
        logits = self.output_layer(x)  # [batch_size, seq_len, num_items+1]
        
        return logits


# ╔═══════════════════════════════════════════════════════════════════════╗
# ║                    ФУНКЦИИ МАСКИРОВАНИЯ И DATASET                     ║
# ╚═══════════════════════════════════════════════════════════════════════╝

def mask_sequence(sequence, mask_token, mask_ratio=0.2):
    """
    Маскирование последовательности для обучения в стиле BERT
    
    Параметры:
    ----------
    sequence : list
        Исходная последовательность товаров
    mask_token : int
        Индекс MASK токена
    mask_ratio : float
        Доля элементов для маскирования (по умолчанию 20%)
        
    Возвращает:
    -----------
    input_seq : list
        Последовательность с замаскированными элементами
    target_seq : list
        Целевые значения (-100 для немаскированных позиций)
    """
    
    # Копируем исходную последовательность
    input_seq = sequence.copy()
    
    # Инициализируем target: -100 означает "игнорировать при расчете loss"
    target_seq = [-100] * len(sequence)

    # ===== ВЫБОР ПОЗИЦИЙ ДЛЯ МАСКИРОВАНИЯ =====
    n_mask = max(1, int(len(sequence) * mask_ratio))  # Минимум 1 маска
    mask_indices = np.random.choice(len(sequence), n_mask, replace=False)
    """
    Случайно выбираем n_mask позиций без повторений
    Это симулирует задачу: "угадай, какой товар был на этой позиции"
    """

    # ===== ПРИМЕНЕНИЕ МАСКИРОВАНИЯ =====
    for idx in mask_indices:
        target_seq[idx] = sequence[idx]  # Сохраняем оригинальный товар как target
        input_seq[idx] = mask_token      # Заменяем на MASK в input
    """
    Модель будет видеть MASK и пытаться предсказать, 
    какой товар был на этом месте, используя контекст
    """

    return input_seq, target_seq


class SequenceDataset(Dataset):
    """
    PyTorch Dataset для последовательностей действий пользователей
    
    Особенности:
    - Автоматически обрезает длинные последовательности
    - Применяет маскирование для каждого примера
    - Добавляет padding для выравнивания длины
    """
    
    def __init__(self, user_seqs, mask_token, max_len):
        """
        Параметры:
        ----------
        user_seqs : dict
            Словарь {user_id: [item1, item2, ...]}
        mask_token : int
            Индекс MASK токена
        max_len : int
            Максимальная длина последовательности
        """
        self.user_seqs = list(user_seqs.values())  # Преобразуем в список
        self.mask_token = mask_token
        self.max_len = max_len

    def __len__(self):
        """Количество пользователей в датасете"""
        return len(self.user_seqs)

    def __getitem__(self, idx):
        """
        Получение одного примера для обучения
        
        Возвращает:
        -----------
        input_seq : torch.Tensor
            Последовательность с масками и padding
        target_seq : torch.Tensor  
            Целевые значения для маскированных позиций
        """
        
        # ===== ОБРЕЗКА ПОСЛЕДОВАТЕЛЬНОСТИ =====
        seq = self.user_seqs[idx][-self.max_len:]
        """
        Берем последние max_len элементов
        Это сохраняет самую свежую историю пользователя
        """
        
        # ===== МАСКИРОВАНИЕ =====
        input_seq, target_seq = mask_sequence(seq, self.mask_token)
        
        # ===== PADDING =====
        pad_len = self.max_len - len(input_seq)
        input_seq = [0] * pad_len + input_seq      # PAD токены в начало
        target_seq = [-100] * pad_len + target_seq  # -100 для игнорирования
        """
        Выравниваем все последовательности до max_len
        Padding добавляется в начало (left padding)
        """

        return torch.tensor(input_seq), torch.tensor(target_seq)


# ╔═══════════════════════════════════════════════════════════════════════╗
# ║                         ФУНКЦИЯ ОБУЧЕНИЯ                              ║
# ╚═══════════════════════════════════════════════════════════════════════╝

def train(model, dataloader, optimizer, device):
    """
    Одна эпоха обучения модели
    
    Параметры:
    ----------
    model : BERT4Rec
        Модель для обучения
    dataloader : DataLoader
        Загрузчик данных с батчами
    optimizer : torch.optim.Optimizer
        Оптимизатор (например, Adam)
    device : torch.device
        Устройство для вычислений (CPU/GPU)
        
    Возвращает:
    -----------
    avg_loss : float
        Средний loss за эпоху
    """
    
    # Переводим модель в режим обучения (включает dropout)
    model.train()
    total_loss = 0
    
    # ===== LOSS ФУНКЦИЯ =====
    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
    """
    CrossEntropyLoss с ignore_index=-100:
    - Вычисляет loss только для маскированных позиций
    - Игнорирует позиции с target=-100 (немаскированные и padding)
    """

    # ===== ЦИКЛ ОБУЧЕНИЯ =====
    for input_seq, target_seq in tqdm(dataloader, desc="Training"):
        # Перемещаем данные на нужное устройство
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        # Обнуляем градиенты от предыдущей итерации
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(input_seq)  # [batch_size, seq_len, vocab_size]

        # ===== ВЫЧИСЛЕНИЕ LOSS =====
        loss = loss_fn(
            logits.view(-1, logits.size(-1)),  # [batch*seq_len, vocab_size]
            target_seq.view(-1)                 # [batch*seq_len]
        )
        """
        Reshape для loss функции:
        - logits: 3D → 2D (объединяем batch и sequence dimensions)
        - targets: 2D → 1D
        
        Loss вычисляется только там, где target != -100
        """
        
        # Backward pass
        loss.backward()
        
        # Обновление весов
        optimizer.step()

        # Накапливаем loss для статистики
        total_loss += loss.item()
    
    # Возвращаем средний loss
    return total_loss / len(dataloader)


# ╔═══════════════════════════════════════════════════════════════════════╗
# ║                    ФУНКЦИЯ ГЕНЕРАЦИИ РЕКОМЕНДАЦИЙ                     ║
# ╚═══════════════════════════════════════════════════════════════════════╝

def recommend_top_k(model, user_seqs, k, mask_token, max_len, device):
    """
    Генерация топ-K рекомендаций для каждого пользователя
    
    Стратегия: добавляем MASK токен в конец истории пользователя
    и предсказываем, какой товар должен быть на этом месте
    
    Параметры:
    ----------
    model : BERT4Rec
        Обученная модель
    user_seqs : dict
        Словарь {user_id: [item1, item2, ...]}
    k : int
        Количество рекомендаций для каждого пользователя
    mask_token : int
        Индекс MASK токена
    max_len : int
        Максимальная длина последовательности
    device : torch.device
        Устройство для вычислений
        
    Возвращает:
    -----------
    recs_df : pd.DataFrame
        DataFrame с колонками ['user', 'item', 'score']
    """
    
    # Переводим модель в режим оценки (отключает dropout)
    model.eval()
    recs = []
    
    # ===== ОБРАБОТКА КАЖДОГО ПОЛЬЗОВАТЕЛЯ =====
    for uid, seq in user_seqs.items():
        # Подготовка последовательности
        seq = seq[-(max_len - 1):]  # Оставляем место для MASK токена
        """
        Берем последние (max_len-1) товаров
        Это нужно, чтобы после добавления MASK длина была ≤ max_len
        """
        
        # Padding + добавление MASK в конец
        pad_len = max_len - 1 - len(seq)
        seq = [0] * pad_len + seq + [mask_token]
        """
        Структура финальной последовательности:
        [PAD, PAD, ..., item1, item2, ..., itemN, MASK]
        """
        
        # Конвертация в тензор
        input_seq = torch.tensor(seq).unsqueeze(0).to(device)  # [1, max_len]

        # ===== ИНФЕРЕНС =====
        with torch.no_grad():  # Отключаем вычисление градиентов
            logits = model(input_seq)     # [1, seq_len, vocab_size]
            scores = logits[0, -1]        # Берем логиты последней позиции (MASK)
            """
            scores содержит "уверенность" модели для каждого товара
            Чем выше score, тем вероятнее этот товар следующий
            """
            
            # Находим топ-K товаров
            topk_values, topk_indices = torch.topk(scores, k)
            topk_items = topk_indices.cpu().tolist()
            
            # ===== ФОРМИРОВАНИЕ РЕЗУЛЬТАТА =====
            for i, item_id in enumerate(topk_items):
                recs.append({
                    'user': uid,
                    'item': item_id,
                    'score': scores[item_id].item()
                })

    return pd.DataFrame(recs)


# ╔═══════════════════════════════════════════════════════════════════════╗
# ║                    ОЦЕНКА МЕТРИК С ПОМОЩЬЮ RECTOOLS                   ║
# ╚═══════════════════════════════════════════════════════════════════════╝

def evaluate_with_rectools(recs_df, ground_truth_df):
    """
    Вычисление метрик качества рекомендаций
    
    Параметры:
    ----------
    recs_df : pd.DataFrame
        Рекомендации с колонками ['user', 'item', 'score']
    ground_truth_df : pd.DataFrame
        Реальные взаимодействия с колонками ['UserID', 'MovieID']
        
    Возвращает:
    -----------
    result : dict
        Словарь с вычисленными метриками
    """
    
    # ===== ПОДГОТОВКА ДАННЫХ =====
    rec_dataset = RecDataset(
        interactions=ground_truth_df.rename(columns={
            'UserID': 'user', 
            'MovieID': 'item'
        }),
        user_features_df=None,
        item_features_df=None
    )
    """
    Создаем объект Dataset из rectools
    Переименовываем колонки для совместимости с библиотекой
    """

    # ===== ОПРЕДЕЛЕНИЕ МЕТРИК =====
    metrics = [
        Recall(k=10),     # Полнота: какая доля релевантных товаров найдена
        Precision(k=10),  # Точность: какая доля рекомендаций релевантна
        NDCG(k=10)        # Ранжирование: учитывает порядок рекомендаций
    ]
    """
    Метрики @ 10:
    - Recall@10: из всех товаров, которые понравились пользователю,
                 какую долю мы включили в топ-10 рекомендаций?
    - Precision@10: из 10 рекомендованных товаров, 
                    какая доля действительно понравилась пользователю?
    - NDCG@10: насколько хорошо мы ранжируем товары?
               (более релевантные должны быть выше в списке)
    """
    
    # ===== ВЫЧИСЛЕНИЕ МЕТРИК =====
    result = calc_metrics(
        metrics=metrics,
        recommendations=recs_df,
        interactions=rec_dataset.interactions
    )
    
    return result



In [2]:
import pandas as pd
import numpy as np
import pickle
import torch
from torch.utils.data import DataLoader
from collections import defaultdict
from sklearn.model_selection import train_test_split

# ===== ЗАГРУЗКА ДАННЫХ =====
print("📁 Загрузка данных MovieLens...")

movies = pd.read_csv('data/bert4rec/movies.dat',
                     sep='::',
                     header=None,
                     names=['MovieID', 'Title', 'Genres'],
                     engine='python',
                     encoding='latin-1')

ratings = pd.read_csv('data/bert4rec/ratings.dat',
                      sep='::',
                      header=None,
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
                      engine='python')

with open('data/bert4rec/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

print(f"✅ Загружено {len(movies)} фильмов")
print(f"✅ Загружено {len(ratings)} оценок")
print(f"✅ Загружено {len(train_data['train'])} пользователей\n")

# ===== АНАЛИЗ ДАННЫХ =====
print("📊 Статистика датасета:")
print(f"Количество пользователей: {ratings['UserID'].nunique()}")
print(f"Количество фильмов: {ratings['MovieID'].nunique()}")
print(f"Средняя длина истории: {np.mean([len(seq) for seq in train_data['train'].values()]):.1f}")
print(f"Максимальная длина истории: {max([len(seq) for seq in train_data['train'].values()])}")

# ===== ПОДГОТОВКА ДАННЫХ ДЛЯ BERT4REC =====

class MovieLensPreprocessor:
    """Препроцессор для подготовки данных MovieLens к BERT4Rec"""
    
    def __init__(self, min_seq_len=5, min_item_freq=5):
        self.min_seq_len = min_seq_len
        self.min_item_freq = min_item_freq
        self.item2id = {}
        self.id2item = {}
        
    def fit_transform(self, sequences):
        """
        Преобразует MovieID в последовательные индексы для модели
        """
        # Подсчет частоты фильмов
        item_counts = defaultdict(int)
        for seq in sequences.values():
            for item in seq:
                item_counts[item] += 1
        
        # Фильтрация редких фильмов
        frequent_items = {item for item, count in item_counts.items() 
                         if count >= self.min_item_freq}
        
        # Создание маппинга MovieID -> index (начиная с 1, 0 = PAD)
        sorted_items = sorted(frequent_items)
        self.item2id = {item: idx + 1 for idx, item in enumerate(sorted_items)}
        self.id2item = {idx: item for item, idx in self.item2id.items()}
        
        # Преобразование последовательностей
        processed_sequences = {}
        for user_id, seq in sequences.items():
            # Фильтруем и преобразуем
            new_seq = [self.item2id[item] for item in seq 
                      if item in self.item2id]
            
            # Оставляем только достаточно длинные последовательности
            if len(new_seq) >= self.min_seq_len:
                processed_sequences[user_id] = new_seq
                
        return processed_sequences
    
    def movie_id_to_title(self, movie_id, movies_df):
        """Получить название фильма по ID"""
        original_id = self.id2item.get(movie_id, None)
        if original_id:
            title = movies_df[movies_df['MovieID'] == original_id]['Title'].values
            return title[0] if len(title) > 0 else f"Movie {original_id}"
        return "Unknown"

# Применяем препроцессинг
preprocessor = MovieLensPreprocessor(min_seq_len=5, min_item_freq=10)
user_sequences = preprocessor.fit_transform(train_data['train'])

print(f"\n✅ После препроцессинга:")
print(f"Количество пользователей: {len(user_sequences)}")
print(f"Количество уникальных фильмов: {len(preprocessor.item2id)}")
print(f"Пример последовательности: {list(user_sequences.values())[0][:5]}...")

# ===== РАЗДЕЛЕНИЕ НА TRAIN/VAL/TEST =====

def split_sequences_for_bert4rec(sequences, val_ratio=0.1, test_ratio=0.1):
    """
    Разделяет последовательности для обучения BERT4Rec
    
    Для каждого пользователя:
    - train: вся последовательность для маскированного обучения
    - val: последний элемент для валидации
    - test: предпоследний элемент для тестирования
    """
    train_seqs = {}
    val_data = []
    test_data = []
    
    for user_id, seq in sequences.items():
        if len(seq) < 3:  # Нужно минимум 3 элемента
            continue
            
        # Для обучения используем всю последовательность (будем маскировать)
        train_seqs[user_id] = seq[:-2]
        
        # Для валидации - предпоследний элемент
        val_data.append({
            'user': user_id,
            'item': seq[-2],
            'history': seq[:-2]
        })
        
        # Для теста - последний элемент
        test_data.append({
            'user': user_id,
            'item': seq[-1],
            'history': seq[:-1]
        })
    
    return train_seqs, val_data, test_data

train_seqs, val_data, test_data = split_sequences_for_bert4rec(user_sequences)
print(f"\n📂 Разделение данных:")
print(f"Train sequences: {len(train_seqs)}")
print(f"Validation: {len(val_data)}")
print(f"Test: {len(test_data)}")

# ===== ИНИЦИАЛИЗАЦИЯ И ОБУЧЕНИЕ BERT4REC =====

# Параметры модели
num_items = len(preprocessor.item2id)
mask_token = num_items + 1  # Последний индекс для MASK
max_seq_len = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"\n🤖 Инициализация BERT4Rec:")
print(f"Количество товаров: {num_items}")
print(f"MASK token ID: {mask_token}")
print(f"Устройство: {device}")

# Создаем модель
model = BERT4Rec(
    num_items=num_items,
    max_seq_len=max_seq_len,
    embed_dim=128,
    num_heads=4,
    hidden_dim=512,
    num_layers=2,
    dropout=0.1
).to(device)

# Создаем датасет и даталоадер
dataset = SequenceDataset(
    user_seqs=train_seqs,
    mask_token=mask_token,
    max_len=max_seq_len
)

train_loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    num_workers=0
)

# Оптимизатор и планировщик
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# ===== ОБУЧЕНИЕ =====
print("\n🚀 Начинаем обучение...")

num_epochs = 10
for epoch in range(num_epochs):
    # Обучение
    avg_loss = train(model, train_loader, optimizer, device)
    scheduler.step()
    
    # Валидация каждые 2 эпохи
    if (epoch + 1) % 2 == 0:
        # Генерируем рекомендации для валидации
        val_seqs = {data['user']: data['history'] for data in val_data}
        val_recs = recommend_top_k(
            model, val_seqs, k=10, 
            mask_token=mask_token, 
            max_len=max_seq_len, 
            device=device
        )
        
        # Подготавливаем ground truth
        val_ground_truth = pd.DataFrame([
            {'user': data['user'], 'item': data['item']} 
            for data in val_data
        ])
        
        # Считаем метрики
        from rectools.metrics import Recall, Precision, HitRate
        
        # Простая метрика Hit Rate@10
        hits = 0
        for data in val_data:
            user_recs = val_recs[val_recs['user'] == data['user']]['item'].values
            if data['item'] in user_recs:
                hits += 1
        hit_rate = hits / len(val_data)
        
        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f} | Val Hit@10: {hit_rate:.4f}")
    else:
        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f}")

# ===== ГЕНЕРАЦИЯ РЕКОМЕНДАЦИЙ =====
print("\n🎯 Генерация финальных рекомендаций...")

# Для тестовых пользователей
test_seqs = {data['user']: data['history'] for data in test_data}
final_recommendations = recommend_top_k(
    model, test_seqs, k=10,
    mask_token=mask_token,
    max_len=max_seq_len,
    device=device
)

# ===== ПРИМЕРЫ РЕКОМЕНДАЦИЙ =====
print("\n🎬 Примеры рекомендаций для первых 3 пользователей:")

for i, user_id in enumerate(list(test_seqs.keys())[:3]):
    print(f"\n👤 Пользователь {user_id}:")
    
    # История просмотров
    history = test_seqs[user_id][-5:]  # Последние 5 фильмов
    print("📜 Последние просмотры:")
    for movie_id in history:
        title = preprocessor.movie_id_to_title(movie_id, movies)
        print(f"   - {title}")
    
    # Рекомендации
    user_recs = final_recommendations[final_recommendations['user'] == user_id]
    print("🎯 Рекомендации:")
    for _, rec in user_recs.head(5).iterrows():
        title = preprocessor.movie_id_to_title(rec['item'], movies)
        print(f"   - {title} (score: {rec['score']:.3f})")

# # ===== СОХРАНЕНИЕ МОДЕЛИ =====
# print("\n💾 Сохранение модели...")
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'preprocessor': preprocessor,
#     'num_items': num_items,
#     'mask_token': mask_token,
#     'max_seq_len': max_seq_len
# }, 'bert4rec_movielens_model.pt')

# print("✅ Модель успешно обучена и сохранена!")

# ===== ФУНКЦИЯ ДЛЯ РЕКОМЕНДАЦИЙ НОВОМУ ПОЛЬЗОВАТЕЛЮ =====

def recommend_for_new_user(movie_titles, model, preprocessor, movies_df, k=10):
    """
    Генерирует рекомендации для нового пользователя на основе списка фильмов
    
    Параметры:
    ----------
    movie_titles : list
        Список названий фильмов, которые смотрел пользователь
    """
    # Находим MovieID по названиям
    movie_ids = []
    for title in movie_titles:
        matches = movies_df[movies_df['Title'].str.contains(title, case=False)]
        if not matches.empty:
            movie_ids.append(matches.iloc[0]['MovieID'])
    
    # Преобразуем в индексы модели
    sequence = [preprocessor.item2id.get(mid, 0) for mid in movie_ids]
    sequence = [idx for idx in sequence if idx > 0]
    
    if not sequence:
        print("❌ Не удалось найти фильмы в базе данных")
        return []
    
    # Генерируем рекомендации
    user_seqs = {'new_user': sequence}
    recs = recommend_top_k(
        model, user_seqs, k=k,
        mask_token=mask_token,
        max_len=max_seq_len,
        device=device
    )
    
    # Преобразуем обратно в названия
    recommendations = []
    for _, rec in recs.iterrows():
        title = preprocessor.movie_id_to_title(rec['item'], movies_df)
        recommendations.append((title, rec['score']))
    
    return recommendations

# Пример использования
print("\n🆕 Рекомендации для нового пользователя:")
user_history = ['Toy Story', 'Star Wars', 'Matrix']
print(f"История: {user_history}")
recs = recommend_for_new_user(user_history, model, preprocessor, movies, k=5)
print("Рекомендации:")
for title, score in recs:
    print(f"  - {title} (score: {score:.3f})")

📁 Загрузка данных MovieLens...
✅ Загружено 3883 фильмов
✅ Загружено 1000209 оценок
✅ Загружено 6040 пользователей

📊 Статистика датасета:
Количество пользователей: 6040
Количество фильмов: 3706
Средняя длина истории: 163.5
Максимальная длина истории: 2275

✅ После препроцессинга:
Количество пользователей: 6040
Количество уникальных фильмов: 3255
Пример последовательности: [1, 2, 3, 4, 5]...

📂 Разделение данных:
Train sequences: 6040
Validation: 6040
Test: 6040

🤖 Инициализация BERT4Rec:
Количество товаров: 3255
MASK token ID: 3256
Устройство: cpu

🚀 Начинаем обучение...


Training: 100%|██████████| 48/48 [00:23<00:00,  2.01it/s]


Epoch 1/10 | Loss: 7.6257


Training: 100%|██████████| 48/48 [00:23<00:00,  2.03it/s]
/opt/homebrew/Caskroom/miniforge/base/envs/recsys/lib/python3.10/site-packages/torch/nn/modules/transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1729647065806/work/aten/src/ATen/NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(


Epoch 2/10 | Loss: 7.4768 | Val Hit@10: 0.0225


...

Epoch 8/10 | Loss: 7.0872 | Val Hit@10: 0.0275


Training: 100%|██████████| 48/48 [00:25<00:00,  1.89it/s]


Epoch 9/10 | Loss: 7.0726


Training: 100%|██████████| 48/48 [00:25<00:00,  1.88it/s]


Epoch 10/10 | Loss: 7.0626 | Val Hit@10: 0.0275

🎯 Генерация финальных рекомендаций...

🎬 Примеры рекомендаций для первых 3 пользователей:

👤 Пользователь 1:
📜 Последние просмотры:
   - How to Make an American Quilt (1995)
   - Seven (Se7en) (1995)
   - Pocahontas (1995)
   - When Night Is Falling (1995)
   - Usual Suspects, The (1995)
🎯 Рекомендации:
   - Juror, The (1996) (score: 3.800)
   - Dracula: Dead and Loving It (1995) (score: 3.764)
   - Big Green, The (1995) (score: 3.597)
   - Georgia (1995) (score: 3.498)
   - Lawnmower Man 2: Beyond Cyberspace (1996) (score: 3.468)

👤 Пользователь 2:
📜 Последние просмотры:
   - First Knight (1995)
   - Free Willy 2: The Adventure Home (1995)
   - Hackers (1995)
   - Jeffrey (1995)
   - Johnny Mnemonic (1995)
🎯 Рекомендации:
   - Devil in a Blue Dress (1995) (score: 3.404)
   - Nine Months (1995) (score: 3.369)
   - Beyond Rangoon (1995) (score: 3.142)
   - Awfully Big Adventure, An (1995) (score: 3.130)
   - Down Periscope (1996) (score: 

In [7]:
# Простая оценка без внешних библиотек
def simple_evaluate(model, test_data, mask_token, max_seq_len, device, k=10):
    """Простая оценка с базовыми метриками"""
    
    # Генерируем рекомендации
    test_seqs = {data['user']: data['history'] for data in test_data}
    recommendations = recommend_top_k(
        model, test_seqs, k=k,
        mask_token=mask_token,
        max_len=max_seq_len,
        device=device
    )
    
    # Считаем Hit Rate
    hits = 0
    for data in test_data:
        user_recs = recommendations[recommendations['user'] == data['user']]['item'].values
        if data['item'] in user_recs:
            hits += 1
    
    hit_rate = hits / len(test_data)
    print(f"Hit Rate@{k}: {hit_rate:.4f}")
    
    return hit_rate

# Использование
hit_rate = simple_evaluate(model, test_data, mask_token, max_seq_len, device, k=10)

Hit Rate@10: 0.0295
